<a href="https://colab.research.google.com/github/HarshadaRaut234/Predicton-of-DR-using-XAI-and-CNN/blob/main/DRprediction_intern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import zipfile
import os
import random
import shutil
from PIL import Image, ImageEnhance, ImageOps

In [4]:
# Path to the ZIP file in Google Drive
zip_file_path = '/content/drive/MyDrive/archive.zip'

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/extracted_images')


In [5]:
# Count images in each folder (class)
class_counts = {}
for folder_name in os.listdir('/content/extracted_images/dr_unified_v2/dr_unified_v2/train'):
    folder_path = os.path.join('/content/extracted_images/dr_unified_v2/dr_unified_v2/train', folder_name)
    if os.path.isdir(folder_path):
        num_images = len(os.listdir(folder_path))
        class_counts[folder_name] = num_images

# Print the number of images in each class
for class_name, count in class_counts.items():
    print(f'Class {class_name}: {count} images')


Class 2: 12119 images
Class 3: 1006 images
Class 1: 3706 images
Class 4: 2002 images
Class 0: 55162 images


In [6]:
# Define the target number of images for each class (e.g., 2000 images per class)
target_num_images = 2000  # Change to 2000 for all classes except Class 3

# Create a new directory to store the downsampled dataset if it doesn't exist
downsampled_dir = '/content/downsampled_images'
if not os.path.exists(downsampled_dir):
    os.makedirs(downsampled_dir)
else:
    # Remove existing images to start fresh
    shutil.rmtree(downsampled_dir)
    os.makedirs(downsampled_dir)
# Function for image augmentation
def augment_image(image):
    # Randomly apply various augmentations
    if random.random() > 0.5:
        image = ImageOps.mirror(image)
    if random.random() > 0.5:
        image = ImageOps.flip(image)
    if random.random() > 0.5:
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(random.uniform(0.5, 1.5))
    if random.random() > 0.5:
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(random.uniform(0.5, 1.5))
    if random.random() > 0.5:
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(random.uniform(0.5, 1.5))
    return image


In [7]:
# Check if the class_counts dictionary is not empty
if class_counts:
    # Iterate over each class and copy exactly 2000 images to the downsampled dataset
    for class_name, num_images in class_counts.items():
        print(f'Processing class {class_name} with {num_images} images')  # Debugging statement
        class_dir = os.path.join('/content/extracted_images/dr_unified_v2/dr_unified_v2/train', class_name)
        if os.path.isdir(class_dir):
            # Create the directory for the downsampled dataset
            os.makedirs(os.path.join(downsampled_dir, class_name), exist_ok=True)

            if class_name == '3':
                # For Class 3, copy all images and then perform augmentation to reach 2000
                sampled_images = os.listdir(class_dir)
            else:
                # For other classes, randomly sample exactly 2000 images
                sampled_images = random.sample(os.listdir(class_dir), target_num_images)

            # Copy sampled images to the downsampled dataset directory
            for image_name in sampled_images:
                src_path = os.path.join(class_dir, image_name)
                dst_path = os.path.join(downsampled_dir, class_name, image_name)
                if os.path.isfile(src_path):  # Check if the item is a file
                    shutil.copy(src_path, dst_path)

            # Ensure the downsampling for other classes
            if class_name != '3':
                existing_images = os.listdir(os.path.join(downsampled_dir, class_name))
                current_count = len(existing_images)
                if current_count > target_num_images:
                    extra_images = random.sample(existing_images, current_count - target_num_images)
                    for image_name in extra_images:
                        os.remove(os.path.join(downsampled_dir, class_name, image_name))

            # Augment images for Class 3 if needed
            if class_name == '3':
                existing_images = os.listdir(os.path.join(downsampled_dir, class_name))
                current_count = len(existing_images)
                while current_count < target_num_images:
                    for image_name in existing_images:
                        if current_count >= target_num_images:
                            break
                        src_path = os.path.join(downsampled_dir, class_name, image_name)
                        image = Image.open(src_path)
                        augmented_image = augment_image(image)
                        augmented_image_name = f'aug_{current_count}.png'
                        augmented_image_path = os.path.join(downsampled_dir, class_name, augmented_image_name)
                        augmented_image.save(augmented_image_path)
                        current_count += 1

    print('Dataset downsampling and upsampling completed successfully.')

    # Count images in each class after downsampling and upsampling
    downsampled_class_counts = {}
    for class_name in os.listdir(downsampled_dir):
        class_path = os.path.join(downsampled_dir, class_name)
        if os.path.isdir(class_path):
            num_images = len(os.listdir(class_path))
            downsampled_class_counts[class_name] = num_images

    # Print the number of images in each class after downsampling and upsampling
    print('\nNumber of images in each class after downsampling and upsampling:')
    for class_name, num_images in downsampled_class_counts.items():
        print(f'Class {class_name}: {num_images} images')
else:
    print('Error: class_counts dictionary is empty. Make sure it is populated before running this code.')


Processing class 2 with 12119 images
Processing class 3 with 1006 images
Processing class 1 with 3706 images
Processing class 4 with 2002 images
Processing class 0 with 55162 images
Dataset downsampling and upsampling completed successfully.

Number of images in each class after downsampling and upsampling:
Class 2: 2000 images
Class 3: 2000 images
Class 1: 2000 images
Class 4: 2000 images
Class 0: 2000 images


In [8]:
from sklearn.model_selection import train_test_split
import shutil

# Define the path to save the split data
split_data_dir = '/content/split_data'
os.makedirs(split_data_dir, exist_ok=True)

# Define the ratio for the train-test-validation split
train_ratio = 0.7
test_ratio = 0.2
val_ratio = 0.1

# Function to split data for each class
def split_data_for_class(class_name):
    class_dir = os.path.join(downsampled_dir, class_name)
    if os.path.isdir(class_dir):
        # List all images
        images = os.listdir(class_dir)
        total_images = len(images)

        # Calculate the number of images for each split
        train_count = int(total_images * train_ratio)
        test_count = int(total_images * test_ratio)
        val_count = total_images - train_count - test_count

        # Create directories for train, test, and validation sets
        train_dir = os.path.join(split_data_dir, 'train', class_name)
        test_dir = os.path.join(split_data_dir, 'test', class_name)
        val_dir = os.path.join(split_data_dir, 'val', class_name)
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(test_dir, exist_ok=True)
        os.makedirs(val_dir, exist_ok=True)

        # Split the data
        train_images, test_val_images = train_test_split(images, test_size=(test_count + val_count), random_state=42)
        test_images, val_images = train_test_split(test_val_images, test_size=(val_count / (test_count + val_count)), random_state=42)

        # Copy images to the respective directories
        for image_name in train_images:
            src_path = os.path.join(class_dir, image_name)
            dst_path = os.path.join(train_dir, image_name)
            shutil.copy(src_path, dst_path)

        for image_name in test_images:
            src_path = os.path.join(class_dir, image_name)
            dst_path = os.path.join(test_dir, image_name)
            shutil.copy(src_path, dst_path)

        for image_name in val_images:
            src_path = os.path.join(class_dir, image_name)
            dst_path = os.path.join(val_dir, image_name)
            shutil.copy(src_path, dst_path)

        print(f'Data split for Class {class_name}')

        # Print the number of images in training, testing, and validation sets for each class
        print(f'Class {class_name}:')
        print(f'  Training set: {len(train_images)} images')
        print(f'  Testing set: {len(test_images)} images')
        print(f'  Validation set: {len(val_images)} images')

# Split data for each class
for class_name in downsampled_class_counts.keys():
    split_data_for_class(class_name)

# Print confirmation
print('Data splitting completed successfully.')


Data split for Class 2
Class 2:
  Training set: 1400 images
  Testing set: 400 images
  Validation set: 200 images
Data split for Class 3
Class 3:
  Training set: 1400 images
  Testing set: 400 images
  Validation set: 200 images
Data split for Class 1
Class 1:
  Training set: 1400 images
  Testing set: 400 images
  Validation set: 200 images
Data split for Class 4
Class 4:
  Training set: 1400 images
  Testing set: 400 images
  Validation set: 200 images
Data split for Class 0
Class 0:
  Training set: 1400 images
  Testing set: 400 images
  Validation set: 200 images
Data splitting completed successfully.
